In [21]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
from PIL import Image
import torch

import numpy as np

from torch import nn
import numpy as np
import matplotlib.pyplot as plt

import requests

import geopandas as gpd
import os

import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


In [22]:
# color palette to map each class to a RGB value
"""{0: 'road', 1: 'sidewalk', 2: 'building', 3: 'wall', 4: 'fence', 5: 'pole', 6: 'traffic light', 7: 'traffic sign', 8: 'vegetation', 9: 'terrain', 10: 'sky', 11: 'person', 12: 'rider', 13: 'car', 14: 'truck', 15: 'bus', 16: 'train', 17: 'motorcycle', 18: 'bicycle'}"""
color_palette = [
    [128, 64, 128],  # road - maroon
    [244, 35, 232],  # sidewalk - pink
    [70, 70, 70],  # building - dark gray
    [102, 102, 156],  # wall - purple
    [190, 153, 153],  # fence - light brown
    [153, 153, 153],  # pole - gray
    [250, 170, 30],  # traffic light - orange
    [220, 220, 0],  # traffic sign - yellow
    [107, 142, 35],  # vegetation - dark green
    [152, 251, 152],  # terrain - light green
    [70, 130, 180],  # sky - blue
    [220, 20, 60],  # person - red
    [255, 0, 0],  # rider - bright red
    [0, 0, 142],  # car - dark blue
    [0, 0, 70],  # truck - navy blue
    [0, 60, 100],  # bus - dark teal
    [0, 80, 100],  # train - dark green
    [0, 0, 230],  # motorcycle - blue
    [119, 11, 32]  # bicycle - dark red
]


In [23]:
def get_models():
    processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic")
    model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic")

    return processor, model

In [24]:
def segment_images(image_path, image_id, processor, model, city, path=""):
    image = Image.open(requests.get(image_path, stream=True).raw)

    inputs = processor(images=image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # you can pass them to processor for postprocessing
    seg = processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
    
    color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # height, width, 3
    palette = np.array(color_palette)
    for label, color in enumerate(palette):
        color_seg[seg == label, :] = color

    # Show image + mask
    img = np.array(image) * 0.4 + color_seg * 0.6
    img = img.astype(np.uint8)

    # Save original image
    dir_path = os.path.join(path, "images", city)
    img_path = os.path.join(dir_path, "{}.jpg".format(image_id))
    image.save(img_path)
    
    # Convert numpy array to PIL Image and save masked image
    pil_img = Image.fromarray(img)
    dir_path = os.path.join(path, "segments", city)
    img_path = os.path.join(dir_path, "{}.png".format(image_id))
    pil_img.save(img_path)
    
    return True

In [25]:
# Download images in parallel
def download_image(image_metadata, city, access_token, processor, model, path=""):
    header = {'Authorization': 'OAuth {}'.format(access_token)}

    image_id = image_metadata["properties"]["id"]
    
    url = 'https://graph.mapillary.com/{}?fields=thumb_1024_url'.format(image_id)
    response = requests.get(url, headers=header)
    data = response.json()
    image_url = data["thumb_1024_url"]

    segment_images(image_url, image_id, processor, model, city)

    return image_id

In [26]:
def download_images_for_points(city, access_token, path=""):
    path_to_file="{}data/{}/{} points with features.gpkg".format(path, city, city)
    gdf_features = gpd.read_file(path_to_file)

    # Load cache
    cache_file = os.path.join(path, "data", city, "cache.txt")
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            cache = set([line.strip() for line in f])
    else:
        cache = set()
    
    # Create the directory if it does not exist
    dir_path = os.path.join(path, "images", city)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    dir_path = os.path.join(path, "segments", city)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    processor, model = get_models()

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
  
        for feature in gdf_features["feature"]:
            feature = json.loads(feature)
            image_id = feature["properties"]["id"]
            
            if image_id not in cache:
                futures.append(executor.submit(download_image, feature, city, access_token, processor, model, path))
 
        for future in (pbar:= tqdm(futures, total=len(futures))):
            pbar.set_description(f"Downloading images")
            image_id = future.result()
            cache.add(image_id)
    
    # Save cache
    with open(cache_file, "w") as f:
        for image_id in cache:
            f.write("{}\n".format(image_id))

In [27]:
# Get the roadnetwork of a specific city using OpenStreetMap data
city = "Kampala, Uganda"

# Set access token for mapillary
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf"


In [28]:
download_images_for_points(city, access_token)